In [ ]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import os
import sys

# ==========================================
# 1. SETUP PATH & LOAD DATA
# ==========================================
current_dir = os.getcwd()
if os.path.basename(current_dir) == 'notebook':
    BASE_DIR = os.path.dirname(current_dir)
else:
    BASE_DIR = current_dir

DATA_DIR = os.path.join(BASE_DIR, 'dataset')
FIG_DIR = os.path.join(BASE_DIR, 'figures')
INPUT_FILE = os.path.join(DATA_DIR, 'final_dataset_2echelon.csv')
OUTPUT_HTML = os.path.join(FIG_DIR, 'peta_zona_30_kecamatan.html')

if not os.path.exists(INPUT_FILE):
    print(f"File {INPUT_FILE} belum ada.")
    print("Jalankan preprocessing data (kode sebelumnya) dulu!")
    sys.exit()

print("⏳ Memuat data...")
df = pd.read_csv(INPUT_FILE)
print(f"Data dimuat: {len(df)} titik.")

# ==========================================
# 2. SETUP WARNA UNTUK 30 KECAMATAN
# ==========================================
# Kita generate 30 kode warna HEX yang berbeda agar tiap kecamatan unik
# Menggunakan colormap 'tab20' + 'tab20b' dari matplotlib
n_clusters = 30
colors = list(mcolors.TABLEAU_COLORS.values()) # Ambil warna dasar
# Kalau kurang, tambah lagi dari CSS4_COLORS
more_colors = list(mcolors.CSS4_COLORS.values())
color_palette = colors + more_colors 

# Fungsi pembantu untuk mengambil warna berdasarkan ID Kecamatan
def get_color(kec_id):
    if kec_id < 0: return '#000000' # TPA (Hitam/Merah)
    return color_palette[kec_id % len(color_palette)]

# ==========================================
# 3. BUAT PETA
# ==========================================
# Pusat peta di rata-rata koordinat Bandung
center_lat = df[df['Type']!='TPA']['Lat'].mean()
center_lon = df[df['Type']!='TPA']['Lon'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles='OpenStreetMap')

# Tambahkan Judul di HTML
title_html = '''
     <h3 align="center" style="font-size:16px"><b>Peta Zonasi 30 Pool Kecamatan (2-Echelon VRP)</b></h3>
     '''
m.get_root().html.add_child(folium.Element(title_html))

print("⏳ Sedang memplot titik...")

for idx, row in df.iterrows():
    # Konten Popup
    popup_txt = f"""
    <div style="font-family: Arial; width: 200px;">
        <b>{row['Nama Fasilitas']}</b><br>
        <hr>
        <b>Jenis:</b> {row['Type']}<br>
        <b>Zona ID:</b> {int(row['Kecamatan_ID'])}<br>
        <b>Beban Sampah:</b> {row['Demand_Ton']} Ton<br>
    </div>
    """
    
    # --- A. PLOT TPA (SINK) ---
    if row['Type'] == 'TPA':
        folium.Marker(
            location=[row['Lat'], row['Lon']],
            popup=folium.Popup(popup_txt, max_width=250),
            tooltip="TPA SARIMUKTI",
            icon=folium.Icon(color='red', icon='trash', prefix='fa')
        ).add_to(m)
        
    # --- B. PLOT POOL KECAMATAN (DEPOT) ---
    elif row['Type'] == 'POOL':
        # Kita pakai Marker dengan warna icon biru/hitam
        # Tapi biar kelihatan beda tiap zona, kita akali di tooltip
        folium.Marker(
            location=[row['Lat'], row['Lon']],
            popup=folium.Popup(popup_txt, max_width=250),
            tooltip=f"POOL KECAMATAN {int(row['Kecamatan_ID'])}",
            icon=folium.Icon(color='black', icon='home', prefix='fa')
        ).add_to(m)
        
        # Opsional: Tambahkan lingkaran radius layanan di sekitar Pool
        folium.Circle(
            location=[row['Lat'], row['Lon']],
            radius=1000, # 1 km radius visual
            color=get_color(int(row['Kecamatan_ID'])),
            fill=True,
            fill_opacity=0.1,
            weight=1
        ).add_to(m)

    # --- C. PLOT TPS (SUPPLY / TRANSSHIPMENT NODE) ---
    elif row['Type'] == 'TPS':
        kec_id = int(row['Kecamatan_ID'])
        zone_color = get_color(kec_id)
        
        folium.CircleMarker(
            location=[row['Lat'], row['Lon']],
            radius=5 + (row['Demand_Ton'] * 0.8), # Besar kecil sesuai volume sampah
            color=zone_color,      # Warna garis pinggir (sesuai zona)
            fill=True,
            fill_color=zone_color, # Warna isi (sesuai zona)
            fill_opacity=0.7,
            popup=folium.Popup(popup_txt, max_width=250),
            tooltip=f"{row['Nama Fasilitas']} (Zona {kec_id})"
        ).add_to(m)

# ==========================================
# 4. SIMPAN HASIL
# ==========================================
m.save(OUTPUT_HTML)
print(f" Peta Berhasil Disimpan!")
print(f" Lokasi: {OUTPUT_HTML}")
print(" Buka file HTML tersebut di browser untuk melihat peta interaktif.")

# Tampilkan peta di notebook (jika support)
m

⏳ Memuat data...
✅ Data dimuat: 209 titik.
⏳ Sedang memplot titik...
✅ Peta Berhasil Disimpan!
📄 Lokasi: d:\College\Semester 7\Teknik Riset Operasi\Tubes\tubes_tro\figures\peta_zona_30_kecamatan.html
👉 Buka file HTML tersebut di browser untuk melihat peta interaktif.
